In [ ]:
!pip install -q transformers datasets evaluate accelerate scikit-learn streamlit pyngrok nltk imbalanced-learn st-annotated-text

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.2 MB/s eta 0:00:00


In [ ]:

import os

from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = "/content/drive/MyDrive/CS772Project"
MODELS_PATH = os.path.join(PROJECT_PATH, "models")
os.makedirs(MODELS_PATH, exist_ok=True)

print(f"Working Directory: {PROJECT_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Working Directory: /content/drive/MyDrive/CS772Project


In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import warnings
warnings.filterwarnings('ignore')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
label_map = {
    'No distortion': 0, 'All-or-nothing thinking': 1, 'Overgeneralization': 2,
    'Mental filter': 3, 'Should statements': 4, 'Labeling': 5,
    'Personalization': 6, 'Magnification': 7, 'Emotional Reasoning': 8,
    'Mind Reading': 9, 'Fortune-telling': 10
}
id2label = {v: k for k, v in label_map.items()}

# 1. Load Stage 1 Data (Shreevastava)
try:
    df_stage1 = pd.read_csv(os.path.join(PROJECT_PATH, 'data/Annotated_data.csv'))

    # Preprocessing: Use 'Distorted part' if available, else full text
    df_stage1['text_to_train'] = df_stage1['Distorted part'].fillna(df_stage1['Patient Question'])
    df_stage1 = df_stage1[['text_to_train', 'Dominant Distortion']].dropna()
    df_stage1['Dominant Distortion'] = df_stage1['Dominant Distortion'].str.strip()

    # Filter valid labels
    df_stage1 = df_stage1[df_stage1['Dominant Distortion'].isin(label_map.keys())]
    df_stage1['label'] = df_stage1['Dominant Distortion'].map(label_map)

    print(f"Loaded Stage 1 Data: {len(df_stage1)} samples")

    # Calculate Class Weights for Stage 1 (Fixes Imbalance)
    unique_classes = np.unique(df_stage1['label'])
    computed_weights = compute_class_weight(class_weight="balanced", classes=unique_classes, y=df_stage1['label'])

    # Create full weight tensor (size 11)
    class_weights_tensor = torch.ones(11).to(device)
    for i, label_idx in enumerate(unique_classes):
        class_weights_tensor[label_idx] = float(computed_weights[i])

    # Split Stage 1
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        df_stage1['text_to_train'].tolist(), df_stage1['label'].tolist(),
        test_size=0.15, random_state=42, stratify=df_stage1['label']
    )

except Exception as e:
    print(f"Error loading Annotated_data.csv: {e}")

# 2. Load Stage 2 Data (Severity)
try:
    df_stage2 = pd.read_csv(os.path.join(PROJECT_PATH, 'data/Depression_Severity_Dataset.csv'))
    # Normalize columns
    if 'Text' in df_stage2.columns: df_stage2.rename(columns={'Text': 'text'}, inplace=True)
    if 'Label' in df_stage2.columns: df_stage2.rename(columns={'Label': 'severity'}, inplace=True)

    print(f"Loaded Stage 2 Data: {len(df_stage2)} samples")
except Exception as e:
    print(f"Error loading Severity Dataset: {e}")

Loaded Stage 1 Data: 1597 samples
Loaded Stage 2 Data: 3553 samples


In [ ]:
from huggingface_hub import login
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import torch
import shutil
import os

# --- AUTHENTICATION ---
print("Please enter your Hugging Face token (required for MentalBERT):")
login()

Please enter your Hugging Face token (required for MentalBERT):


In [ ]:
model_checkpoint = "mental/mental-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

class DistortionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = DistortionDataset(train_encodings, train_labels)
val_dataset = DistortionDataset(val_encodings, val_labels)

# --- UPDATED METRICS FUNCTION (Binary + Multi-class) ---
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    # 1. Multi-class Metrics (Specific Distortions 0-10)
    precision_m, recall_m, f1_m, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    f1_macro = f1_score(labels, preds, average='macro')
    acc_m = accuracy_score(labels, preds)

    # 2. Binary Metrics (Distortion [1-10] vs No Distortion [0])
    # Map all classes > 0 to 1 (Distortion), Keep 0 as 0 (Normal)
    bin_preds = np.where(preds > 0, 1, 0)
    bin_labels = np.where(labels > 0, 1, 0)

    precision_b, recall_b, f1_b, _ = precision_recall_fscore_support(bin_labels, bin_preds, average='binary')
    acc_b = accuracy_score(bin_labels, bin_preds)

    return {
        'multi_accuracy': acc_m,
        'multi_f1_weighted': f1_m,
        'multi_f1_macro': f1_macro,
        'multi_precision': precision_m,
        'multi_recall': recall_m,

        'binary_accuracy': acc_b,
        'binary_f1': f1_b,             # <--- This compares to the 82.77 baseline
        'binary_precision': precision_b,
        'binary_recall': recall_b
    }

# Trainer
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=11, id2label=id2label, label2id=label_map
).to(device)

checkpoint_dir = os.path.join(MODELS_PATH, "checkpoints_temp")

training_args = TrainingArguments(
    output_dir=checkpoint_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1,
    logging_steps=50,
    metric_for_best_model="multi_f1_weighted", # Optimization Goal
    report_to="none"
)

trainer = WeightedTrainer(
    model=model, args=training_args,
    train_dataset=train_dataset, eval_dataset=val_dataset,
    tokenizer=tokenizer, compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


print("Starting Stage 1 Training...")
trainer.train()

# --- PRINT DETAILED REPORT ---
print("\n" + "="*40)
print("STAGE 1 EVALUATION (Validation Set)")
print("="*40)
metrics = trainer.evaluate()

print(f"BINARY METRICS (Distortion vs Normal):")
print(f"   Accuracy:  {metrics['eval_binary_accuracy']:.4f}")
print(f"   F1 Score:  {metrics['eval_binary_f1']:.4f}")
print(f"   Precision: {metrics['eval_binary_precision']:.4f}")
print(f"   Recall:    {metrics['eval_binary_recall']:.4f}")
print("-" * 40)
print(f"MULTI-CLASS METRICS (11 Types):")
print(f"   Weighted F1: {metrics['eval_multi_f1_weighted']:.4f}")
print(f"   Macro F1:    {metrics['eval_multi_f1_macro']:.4f}")
print(f"   Accuracy:    {metrics['eval_multi_accuracy']:.4f}")
print("="*40)

# Save & Cleanup
final_model_path = os.path.join(MODELS_PATH, "mentalbert_specialist")
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)
if os.path.exists(checkpoint_dir): shutil.rmtree(checkpoint_dir)
print(f"Model saved to: {final_model_path}")

Starting Stage 1 Training...


Epoch,Training Loss,Validation Loss,Multi Accuracy,Multi F1 Weighted,Multi F1 Macro,Multi Precision,Multi Recall,Binary Accuracy,Binary F1,Binary Precision,Binary Recall
1,2.320300,2.168738,0.266667,0.194540,0.190212,0.176051,0.266667,1.000000,1.000000,1.000000,1.000000
2,1.991000,1.931627,0.362500,0.357271,0.339304,0.399388,0.362500,1.000000,1.000000,1.000000,1.000000
3,1.594800,1.833775,0.391667,0.374994,0.349551,0.414476,0.391667,1.000000,1.000000,1.000000,1.000000
4,1.444600,1.810223,0.370833,0.360017,0.340360,0.381737,0.370833,1.000000,1.000000,1.000000,1.000000
5,1.235200,1.789765,0.395833,0.383299,0.366896,0.402164,0.395833,1.000000,1.000000,1.000000,1.000000



STAGE 1 EVALUATION (Validation Set)


BINARY METRICS (Distortion vs Normal):
   Accuracy:  1.0000
   F1 Score:  1.0000
   Precision: 1.0000
   Recall:    1.0000
----------------------------------------
MULTI-CLASS METRICS (11 Types):
   Weighted F1: 0.3833
   Macro F1:    0.3669
   Accuracy:    0.3958
Model saved to: /content/drive/MyDrive/CS772Project/models/mentalbert_specialist


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import pickle
import nltk
from tqdm import tqdm
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. SETUP & PATHS ---
PROJECT_PATH = "/content/drive/MyDrive/CS772Project"
MODELS_PATH = os.path.join(PROJECT_PATH, "models")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Download VADER for Sentiment Analysis
nltk.download('vader_lexicon', quiet=True)
sia = SentimentIntensityAnalyzer()

print(f"Working Directory: {PROJECT_PATH}")

# --- 2. LOAD DATA & SPECIALIST MODEL ---
print("Loading resources...")
try:
    # Load Severity Data
    df_stage2 = pd.read_csv(os.path.join(PROJECT_PATH, 'data/Depression_Severity_Dataset.csv'))
    if 'Text' in df_stage2.columns: df_stage2.rename(columns={'Text': 'text'}, inplace=True)
    if 'Label' in df_stage2.columns: df_stage2.rename(columns={'Label': 'severity'}, inplace=True)

    # Load Stage 1 Model (The Distortion Specialist)
    specialist_path = os.path.join(MODELS_PATH, "mentalbert_specialist")
    tokenizer = AutoTokenizer.from_pretrained(specialist_path)
    model = AutoModelForSequenceClassification.from_pretrained(specialist_path).to(device)
    model.eval()
    print("Stage 1 Model Loaded.")
except Exception as e:
    print(f"Error: {e}")
    print("Ensure you have uploaded the dataset and trained Stage 1 first.")

# --- 3. HYBRID FEATURE EXTRACTION (16 Features) ---
# We combine BERT Distortion Probabilities (11) + Sentiment/Stats (5)
print("Generating Hybrid Features (Distortions + Sentiment)...")

def extract_features_hybrid(texts, batch_size=32):
    distortion_probs = []
    sentiment_feats = []

    # A. Distortions (11 Features)
    for i in tqdm(range(0, len(texts), batch_size), desc="BERT Inference"):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            distortion_probs.append(probs.cpu().numpy())

    distortion_matrix = np.vstack(distortion_probs)

    # B. Sentiment & Stats (5 Features)
    for text in texts:
        scores = sia.polarity_scores(str(text))
        word_count = len(str(text).split())
        # [Compound, Neg, Neu, Pos, WordCount]
        sentiment_feats.append([scores['compound'], scores['neg'], scores['neu'], scores['pos'], word_count])

    # C. Combine -> 16 Features
    return np.hstack((distortion_matrix, np.array(sentiment_feats)))

X = extract_features_hybrid(df_stage2['text'].tolist())
print(f"Feature Shape: {X.shape} (N samples x 16 features)")

# --- 4. PREPARE LABELS & SPLIT ---
# Handle case variations in labels (Minimum vs minimum)
severity_map_lower = {'minimum': 0, 'mild': 1, 'moderate': 2, 'severe': 3}
severity_map_title = {'Minimum': 0, 'Mild': 1, 'Moderate': 2, 'Severe': 3}

y = df_stage2['label']
if isinstance(y.iloc[0], str):
    # Try mapping, default to title case if lower fails or vice versa
    if y.iloc[0] in severity_map_title:
        y = y.map(severity_map_title)
    else:
        y = y.map(severity_map_lower)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- 5. SMOTE (Fixes Class Imbalance) ---
print(f"Original Train Counts: {np.bincount(y_train)}")
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
print(f"Balanced Train Counts: {np.bincount(y_train_bal)}")

# --- 6. TRAIN GRADIENT BOOSTING ---
print("Training Gradient Boosting Classifier...")
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train_bal, y_train_bal)

# --- 7. EVALUATION & COMPARISON ---
y_pred = gb_model.predict(X_test)
target_names = ['Minimum', 'Mild', 'Moderate', 'Severe']

print("\n" + "="*50)
print("STAGE 2 RESULTS (Hybrid Features + SMOTE)")
print("="*50)
print(classification_report(y_test, y_pred, target_names=target_names))
print("-" * 50)

# Calculate Weighted F1 for Baseline Comparison
report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)
weighted_f1 = report['weighted avg']['f1-score']
print(f"Weighted F1 Score: {weighted_f1:.4f}")
print(f"Baseline to Beat:      0.5800")

if weighted_f1 > 0.58:
    print("STATUS: BASELINE BEATEN")
else:
    print("STATUS: COMPETITIVE (Check individual class recall)")

# --- 8. SAVE MODEL ---
rf_path = os.path.join(MODELS_PATH, "rf_severity_model.pkl")
with open(rf_path, 'wb') as f:
    pickle.dump(gb_model, f)

print(f"\nHybrid Model saved to: {rf_path}")

Working Directory: /content/drive/MyDrive/CS772Project
Loading resources...
Stage 1 Model Loaded.
Generating Hybrid Features (Distortions + Sentiment)...


BERT Inference: 100%|██████████| 112/112 [00:29<00:00,  3.80it/s]


Feature Shape: (3553, 16) (N samples x 16 features)
Original Train Counts: [2069  232  315  226]
Balanced Train Counts: [2069 2069 2069 2069]
Training Gradient Boosting Classifier...

STAGE 2 RESULTS (Hybrid Features + SMOTE)
              precision    recall  f1-score   support

     Minimum       0.86      0.69      0.76       518
        Mild       0.15      0.29      0.20        58
    Moderate       0.26      0.33      0.29        79
      Severe       0.20      0.30      0.24        56

    accuracy                           0.59       711
   macro avg       0.37      0.40      0.37       711
weighted avg       0.68      0.59      0.62       711

--------------------------------------------------
Weighted F1 Score: 0.6243
Baseline to Beat:      0.5800
STATUS: BASELINE BEATEN

Hybrid Model saved to: /content/drive/MyDrive/CS772Project/models/rf_severity_model.pkl


In [ ]:
!pip install -q streamlit st-annotated-text pyngrok nltk xgboost transformers
import nltk
nltk.download('vader_lexicon', quiet=True)
print("Dependencies installed.")

Dependencies installed.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import torch
import torch.nn.functional as F
import numpy as np
import pickle
import os
import nltk
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.sentiment import SentimentIntensityAnalyzer

# --- 1. SETUP & PATHS ---
PROJECT_PATH = "/content/drive/MyDrive/CS772Project"
MODEL_PATH = os.path.join(PROJECT_PATH, "models/mentalbert_specialist")
RF_PATH = os.path.join(PROJECT_PATH, "models/rf_severity_model.pkl")

# Initialize NLTK
try:
    nltk.data.find('vader_lexicon')
except LookupError:
    nltk.download('vader_lexicon', quiet=True)
sia = SentimentIntensityAnalyzer()

# --- 2. LOAD MODELS (Cached) ---
if 'bert_model' not in globals() or 'rf_model' not in globals():
    print("Loading models... (this happens only once)")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
        bert_model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
        with open(RF_PATH, 'rb') as f:
            rf_model = pickle.load(f)
        print("Models Loaded Successfully!")
    except Exception as e:
        print(f"Error loading models: {e}")
        print(f"Checked path: {MODEL_PATH}")
else:
    print("Models already loaded in memory.")

# --- 3. LOGIC ---
id2label = {
    0: 'No Distortion', 1: 'All-or-nothing', 2: 'Overgeneralization',
    3: 'Mental filter', 4: 'Should statements', 5: 'Labeling',
    6: 'Personalization', 7: 'Magnification', 8: 'Emotional Reasoning',
    9: 'Mind Reading', 10: 'Fortune-telling'
}

def analyze_text(text):
    # A. Guardrails (Filter casual chat)
    scores = sia.polarity_scores(text)
    word_count = len(text.split())
    is_casual = word_count < 10 and abs(scores['compound']) < 0.2

    # B. BERT Inference (Distortions)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
        dist_probs = F.softmax(outputs.logits, dim=1).cpu().numpy()[0]

    if is_casual:
        # Override for casual text
        dist_probs = np.zeros_like(dist_probs)
        dist_probs[0] = 0.99 # No Distortion

    # C. Hybrid Feature Construction
    extra_feats = np.array([scores['compound'], scores['neg'], scores['neu'], scores['pos'], word_count])
    hybrid_features = np.hstack((dist_probs, extra_feats)).reshape(1, -1)

    # D. Severity Prediction (With Risk-Sensitive Thresholding)
    severity_proba = rf_model.predict_proba(hybrid_features)[0]

    # Default to Argmax
    severity_pred = np.argmax(severity_proba)

    # Apply Sensitive Thresholds (Medical AI Logic: Avoiding to miss high risk)
    # Proba Indices: 0=Minimum, 1=Mild, 2=Moderate, 3=Severe
    if not is_casual:
        if severity_proba[3] > 0.20:   # If Severe risk > 20%, flag it
            severity_pred = 3
        elif severity_proba[2] > 0.30: # If Moderate risk > 30%, flag it
            severity_pred = 2
        elif severity_proba[1] > 0.35: # If Mild risk > 35%, flag it
            severity_pred = 1

    if is_casual: severity_pred = 0 # Force Minimum

    labels = ['Minimum', 'Mild', 'Moderate', 'Severe']
    risk_label = labels[severity_pred]
    confidence = severity_proba[severity_pred]

    # E. Formulate Output
    # 1. Distortion Logic
    prob_distorted = 1.0 - dist_probs[0]
    dist_dict = {id2label[i]: float(dist_probs[i]) for i in range(len(dist_probs))}
    dominant_dist = max(dist_dict, key=dist_dict.get)

    # 2. Rationale Logic
    rationale = ""
    if is_casual:
        rationale = "Text identified as casual/neutral conversation. No clinical markers found."
    elif risk_label == "Minimum":
        if scores['neg'] > 0.3:
            rationale = "High negative sentiment detected, but lack of specific cognitive distortion patterns suggests situational distress (sadness) rather than clinical depression."
        else:
            rationale = "Low distortion levels and neutral sentiment indicate healthy cognitive patterns."
    else:
        rationale = f"Risk Classification: **{risk_label}**.<br>Driven by **{dominant_dist}** ({dist_dict[dominant_dist]*100:.1f}%) combined with negative sentiment ({scores['neg']*100:.1f}%)."

    return {
        "is_distorted": "YES" if prob_distorted > 0.5 else "NO",
        "dist_prob": prob_distorted,
        "risk": risk_label,
        "risk_conf": confidence,
        "rationale": rationale,
        "dominant": dominant_dist,
        "profiles": dist_dict
    }

# --- 4. UI WIDGETS ---
text_input = widgets.Textarea(
    value='',
    placeholder='Type here (e.g., "I feel like a failure today.")',
    description='Input:',
    layout=widgets.Layout(width='90%', height='100px')
)
run_btn = widgets.Button(
    description='Analyze Risk',
    button_style='primary',
    icon='search'
)
out = widgets.Output()

def on_click(b):
    with out:
        clear_output()
        if not text_input.value.strip():
            print("Please enter some text.")
            return

        print("Analyzing...")
        res = analyze_text(text_input.value)

        # Define Colors
        risk_color = {
            "Minimum": "green", "Mild": "blue", "Moderate": "orange", "Severe": "red"
        }.get(res['risk'], "black")

        # HTML Output
        html_code = f"""
        <div style="border: 1px solid #ddd; padding: 20px; border-radius: 10px; max-width: 800px;">
            <h2 style="margin-top:0;">Clinical Assessment Report</h2>
            <hr>

            <!-- Top Section: Binary & Risk -->
            <div style="display: flex; justify-content: space-between;">
                <div>
                    <b>Distortion Detected:</b>
                    <span style="font-size: 1.2em; color: {'red' if res['is_distorted']=='YES' else 'green'}">
                        {res['is_distorted']}
                    </span> ({res['dist_prob']*100:.1f}%)
                </div>
                <div style="text-align: right;">
                    <b>Severity Risk:</b>
                    <span style="font-size: 1.5em; color: {risk_color}; font-weight: bold;">
                        {res['risk']}
                    </span>
                    <br><small>Confidence: {res['risk_conf']*100:.1f}%</small>
                </div>
            </div>

            <br>

            <!-- Middle Section: Rationale -->
            <div style="background-color: #f9f9f9; padding: 15px; border-left: 5px solid #333; color: #000">
                <b>Diagnostic Rationale:</b><br>
                {res['rationale']}
            </div>

            <br>

            <!-- Bottom Section: Profile -->
            <b>Cognitive Distortion Profile (Top 3):</b>
            <ul>
        """

        # Add Top 3 Distortions
        sorted_profiles = sorted(res['profiles'].items(), key=lambda x: x[1], reverse=True)
        for name, score in sorted_profiles[:3]:
            if name == "No Distortion": continue
            bar_width = int(score * 300) # Scale for simple CSS bar
            html_code += f"""
                <li style="margin-bottom: 5px;">
                    {name}: <b>{score*100:.1f}%</b>
                    <div style="background-color: {risk_color}; width: {bar_width}px; height: 10px; border-radius: 5px; opacity: 0.7;"></div>
                </li>
            """

        html_code += "</ul></div>"

        display(HTML(html_code))

run_btn.on_click(on_click)

# Display UI
print("Enter Patient Text Below:")
display(text_input, run_btn, out)

Models already loaded in memory.
Enter Patient Text Below:


Textarea(value='', description='Input:', layout=Layout(height='100px', width='90%'), placeholder='Type here (e…

Button(button_style='primary', description='Analyze Risk', icon='search', style=ButtonStyle())

Output()